In [1]:
# Cargamos librerías
import numpy as np # numérico
import pandas as pd # marcos de datos
from sklearn.datasets import load_diabetes # datos de diabetes
from sklearn.pipeline import Pipeline # tuberías de procesamiento
from sklearn.preprocessing import StandardScaler # función de normalización
from sklearn.linear_model import LinearRegression # modelo de regresión lineal
from sklearn.linear_model import Ridge #  modelos de regresión ridge
from sklearn.linear_model import Lasso # modelo de regresión Lasso
from sklearn.model_selection import train_test_split # divisor de datos
from sklearn.metrics import mean_squared_error # función de error cuadrático medio
from sklearn.model_selection import GridSearchCV # validación cruzada en malla

In [2]:
# Cargamos datos de diabetes
diabetes = load_diabetes()

In [3]:
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [4]:
# Hacemos el marco de datos según feature_names
X = pd.DataFrame(diabetes.data,columns = diabetes.feature_names)
y = pd.DataFrame(diabetes.target,columns = ['daiabet_prog'])

In [5]:
# Observamos los features
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [ ]:
# Observamos la dimensión
X.shape

(442, 10)

In [6]:
# Separamos en datos de prueba y entrenamiento
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 111)

In [7]:
# Definimos tubería de procesamiento del modelo lineal
linreg = Pipeline([
    ("scaler", StandardScaler()),         # Normalizamos
    ("regressor", LinearRegression())     # Aplicamos regresión lineal
])

In [8]:
# Ajustamos modelo de regresión lineal
linreg.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', LinearRegression())])

In [9]:
# Evaluamos modelo en datos de prueb
y_lin_pred = linreg.predict(X_test)
loss_lin = mean_squared_error(y_test,y_lin_pred)
loss_lin

2911.5629288662753

In [10]:
y

,daiabet_prog
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0
...,...
437,178.0
438,104.0
439,132.0
440,220.0


In [11]:
# Alternativamente podemos normalizar los targets para que el mse esté en la escala normalizada
from sklearn.compose import TransformedTargetRegressor

In [ ]:
# Modelo con target scaled
linreg_ts = TransformedTargetRegressor(
    regressor = linreg ,
    transformer = StandardScaler()
)

In [ ]:
# Ajustamos regresión lineal target scaled
linreg_ts.fit(X_train, y_train)

TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                      StandardScaler()),
                                                     ('regressor',
                                                      LinearRegression())]),
                           transformer=StandardScaler())

In [ ]:
# Evaluamos modelo en datos de prueba
y_lin_pred_ts = linreg_ts.predict(X_test)
loss_lin_ts = mean_squared_error(y_test,y_lin_pred_ts)
loss_lin_ts

2911.5629288662753

# Regresión ridge

In [ ]:
# Definimos tubería de procesamiento del modelo de regresión ridge
ridreg = Pipeline([
    ("scaler", StandardScaler()),    # Normalizamos
    ("regressor", Ridge(alpha=1.0)) # Aplicamos regresión ridge con parámetro de regularización 1.0
])

In [ ]:
# Ajustamos modelo ridge
ridreg.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Ridge())])

In [ ]:
# Evaluamos modelo
y_rid_pred = ridreg.predict(X_test)
loss_rid = mean_squared_error(y_test,y_rid_pred)
loss_rid

2916.602897605101

In [ ]:
# ¿Cómo elegimos el parámetro de penalización
ridreg = Pipeline([
    ("scaler", StandardScaler()),    # Normalizamos
    ("regressor", Ridge(alpha=10.0)) # Aplicamos regresión ridge con parámetro de regularización 10.0
])
ridreg.fit(X_train,y_train)
y_rid_pred = ridreg.predict(X_test)
loss_rid = mean_squared_error(y_test,y_rid_pred)
loss_rid

2911.0136578507386

In [ ]:
# Validación cruzada
ridreg = Pipeline([
    ("scaler", StandardScaler()),    # Normalizamos
    ("regressor", Ridge()) # Aplicamos regresión ridge con parámetro de regularización 10.0
])
param_grid = {
    'ridge__alpha': [1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]
}
ridreg_cv = GridSearchCV(ridreg,param_grid,scoring='neg_mean_squared_error',cv=5)
ridreg_cv.fit(X_train,y_train)

ValueError: Invalid parameter 'ridge' for estimator Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Ridge())]). Valid parameters are: ['memory', 'steps', 'transform_input', 'verbose'].

In [ ]:
# Revisamos cuál es el mejor modelo y su score
print(ridreg_cv.best_params_)
print(ridreg_cv.best_score_)

{'alpha': 0.01}
-3105.058879124681


In [ ]:
# Procedemos análogamente para Lasso

In [ ]:
lasso_reg = Lasso(alpha=0.01) # selecting ridge and best parameters
lasso_reg.fit(X_train,y_train)

Lasso(alpha=0.01)

In [ ]:
lassoreg = Pipeline([
    ("scaler", StandardScaler()),    # Normalizamos
    ("regressor", Lasso()) # Aplicamos regresión ridge con parámetro de regularización 10.0
])
alphas = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridreg_cv = GridSearchCV(rid_reg,alphas,scoring='neg_mean_squared_error',cv=5)
ridreg_cv.fit(X_train,y_train)

In [ ]:
y_lasso_pred = lasso_reg.predict(X_test)
loss_lasso = mean_squared_error(y_test,y_lasso_pred)
loss_lasso

0.49105687352835

In [ ]:
lasso_reg = Lasso()
lasso_reg_cv = GridSearchCV(lasso_reg,alphas,scoring='neg_mean_squared_error',cv=10)
lasso_reg_cv.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.232e+01, tolerance: 3.135e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.096e+01, tolerance: 3.128e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.034e+01, tolerance: 3.068e

GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 55, 100]},
             scoring='neg_mean_squared_error')

In [ ]:
# Let's check out the best parameter and the score
print(lasso_reg_cv.best_params_)
print(lasso_reg_cv.best_score_)

{'alpha': 0.01}
-0.5158479350084135


In [ ]:
loss_lin < loss_lasso

True